In [4]:
run create_map.py

Label(value='')

In [5]:
my_map

Map(center=[9.700554008484533, -0.541705192919114], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [ ]:
print(year, lat, lon)

In [6]:
meteo_file = get_era5_gee(year, lat, lon, dest_folder="data/ERA5_weather/")

./python/wofost_utils.py:303: RuntimeWarning: Mean of empty slice
  humidity = np.maximum(np.nanmean(para_vals[0], axis=1), 0)
./python/wofost_utils.py:304: RuntimeWarning: All-NaN slice encountered
  temperature_2m_max = np.nanmax(para_vals[1], axis=1)
./python/wofost_utils.py:305: RuntimeWarning: All-NaN slice encountered
  temperature_2m_min = np.nanmin(para_vals[1], axis=1)
./python/wofost_utils.py:308: RuntimeWarning: Mean of empty slice
  wind = np.nanmean(np.sqrt(para_vals[4] ** 2 + para_vals[5] ** 2), axis=1)


In [1]:
mkdir data/ERA5_weather/